<a href="https://colab.research.google.com/github/gabrie1-s/cnn-1d/blob/combined_approaches/Sensible_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from pathlib import Path
import scipy.io
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import pandas as pd
from sklearn.preprocessing import normalize, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from scipy.stats import zscore
import seaborn as sns
from sklearn.metrics import make_scorer, f1_score

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('drive/MyDrive/CNN_Chagas/Physionet/cnn-1d/Resultado_Processamento_v2.csv')

In [4]:
df

,Dur_Sinal(s),#QRS,Media Amp QRS,Std Amp QRS,Median Amp QRS,Media RR (s),Std RR (s),Median RR (s),Media Dur_QRS (s),Std Dur_QRS (s),...,Estat_A8_8,Estat_A8_9,Estat_A8_10,Estat_A8_11,Estat_A8_12,Estat_A8_13,Estat_A8_14,Estat_A8_15,Estat_A8_16,Classe
0,24.600000,33,836.419603,85.831472,831.877704,0.766146,0.033453,0.765000,0.092929,0.010761,...,-26.298908,-21.354659,-8.540299,1.040276,7.025327,16.925916,38.003758,60.524885,80.457494,1
1,24.280000,26,898.558684,379.684805,760.503584,0.968133,0.475049,1.016667,0.089487,0.025345,...,-63.713815,-41.019342,-18.886175,-2.531223,23.868779,36.430681,52.330689,61.675409,86.493207,1
2,54.413333,74,-441.007864,174.339863,-463.457897,0.744247,0.043095,0.740000,0.085811,0.016690,...,-49.505647,-33.130233,-7.611446,-0.331462,10.485241,28.535933,54.983604,78.092660,93.922501,1
3,24.083333,26,1235.090843,112.169859,1258.864700,0.960000,0.188063,0.923333,0.098974,0.015657,...,-21.765775,-11.838527,-5.959067,1.254213,6.580426,17.478032,28.314614,72.372072,110.821600,2
4,52.720000,99,1024.641289,103.292198,1021.379466,0.537041,0.223627,0.443333,0.082357,0.011926,...,-50.898650,-38.675635,-15.309947,1.134889,14.478074,40.776089,51.727968,76.243152,80.596417,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8523,24.350000,29,296.845203,16.603457,295.855678,0.866190,0.027722,0.866667,0.103448,0.013583,...,-24.980985,-16.412939,-7.466233,0.000243,9.653857,16.145173,19.820713,28.646123,29.334085,1
8524,24.030000,36,-839.505476,447.158133,-988.710393,0.682381,0.107660,0.686667,0.136204,0.016565,...,-72.617034,-55.136136,-29.351067,-8.660475,23.340607,72.002703,84.253433,102.269232,115.199885,3
8525,34.836667,43,890.801644,131.515162,925.172333,0.827540,0.063044,0.831667,0.086357,0.010435,...,-57.278665,-40.848680,-22.595483,-0.611382,18.588751,46.321316,74.263442,103.889577,111.960099,1
8526,24.506667,43,450.832147,272.511305,437.161554,0.582698,0.093104,0.606667,0.069767,0.015386,...,-78.475511,-55.234433,-23.792759,4.228913,27.782416,63.342404,82.277468,103.347309,114.517537,1


In [5]:
x = df.iloc[:, 0:-1]
y = df["Classe"]

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, shuffle=False)

In [7]:
y_train = [i-1 for i in y_train]
y_valid = [i-1 for i in y_valid]

In [9]:
import xgboost as xgb


xgb_cl = xgb.XGBClassifier(max_depth=5, gamma=2.0, learning_rate=0.1,
                           subsample=0.8, num_boost_round=100, min_child_weight=7)

# kfold = KFold(n_splits=200, random_state=7, shuffle=True)
# results = cross_val_score(xgb_cl, x_train, y_train, cv=kfold, scoring='f1_macro', n_jobs=6)

xgb_cl.fit(x_train, y_train)

TypeError: ignored

In [ ]:
y_pred = xgb_cl.predict(x_valid)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

plt.rcParams['figure.figsize']=(20,5)
plt.rcParams.update({'font.size': 20})

cf_matrix = confusion_matrix(y_valid, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = ['N', 'A', 'O', '~'])
cm_display.plot()

In [ ]:
from sklearn.metrics import classification_report 
 
relatorio = classification_report(y_valid, y_pred, target_names = ['N', 'A', 'O', '~']) 
print(relatorio)

In [ ]:
np.save("drive/MyDrive/CNN_Chagas/Physionet/cnn-1d/sensible_train", x_train)
np.save("drive/MyDrive/CNN_Chagas/Physionet/cnn-1d/sensible_valid", x_valid)

# Aplicando PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(x)
x_new = scaler.transform(x)    
pca = PCA()
x_new = pca.fit_transform(x_new)

In [ ]:
var = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
var

In [ ]:
for i, v in enumerate(var):
  if v >= 90.:
    break

x_new  = x_new[:, :(i+1)]

In [ ]:
len(x_new[0])

In [ ]:
d = pd.DataFrame(x_new)

In [ ]:
for col in d:
  val  = d[col]/(max(d[col]) - min(d[col]))
  d[col] = val

In [ ]:
x_new = d

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_new, y, test_size=0.15)

In [ ]:
y_train = [i-1 for i in y_train]
y_valid = [i-1 for i in y_valid]

In [ ]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier(max_depth=5, gamma=2.0, learning_rate=0.1,
                           subsample=0.8, num_boost_round=100, min_child_weight=7)
xgb_cl.fit(x_train, y_train)

In [ ]:
y_pred = xgb_cl.predict(x_valid)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

plt.rcParams['figure.figsize']=(20,5)
plt.rcParams.update({'font.size': 20})

cf_matrix = confusion_matrix(y_valid, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cf_matrix, display_labels = ['N', 'A', 'O', '~'])
cm_display.plot()

In [ ]:
from sklearn.metrics import classification_report 
 
relatorio = classification_report(y_valid, y_pred, target_names = ['N', 'A', 'O', '~']) 
print(relatorio)